In [1]:
import pymongo
import pandas as pd
import numpy as np

In [2]:
client = pymongo.MongoClient("localhost",connect=False)

In [3]:
db = client["Zufang-beijing-530"]

In [4]:
table = db["zufang"]

In [5]:
data = pd.DataFrame(list(table.find()))

In [6]:
del data["_id"]
del data["网址"]

In [7]:
df = data.drop_duplicates()

In [8]:
df.tail(3)

,交租方式,地址,建筑面积,户型,更新时间,标签,标题,楼层,租金,装修
120618,面议,门头沟 - 石泉砖厂,65,2室1厅,9秒前,"采光好,交通便利,周边配套齐",石泉龙门两居室 有家具 快四沿线 河滩 交通便利,10/21,3300,简装修
120619,押一付三,朝阳 - 东方雅苑,100,2室2厅,11秒前,"交通便利,周边配套齐,独门独卫",西大望路 东方雅苑二居!多话没有!看图可短租可月付,6/12,7500,豪华装修
120620,押一付三,朝阳 - 苹果社区,55,1室1厅,12秒前,None,苹果艺术街区二装经典南向开间 家装齐全 整洁干净 温馨入住,10/25,6800,精装修


In [9]:
df2 = df.iloc[:,[1,2,3,8]].dropna().drop_duplicates()
df2.tail(3)


,地址,建筑面积,户型,租金
120591,朝阳 - 乐成豪丽,68,1室1厅,11000
120598,海淀 - 上地东里,74,3室1厅,7000
120619,朝阳 - 东方雅苑,100,2室2厅,7500


In [10]:
def trans_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [11]:
df2["建筑面积"] =df2["建筑面积"].apply(trans_int).dropna(how='any').astype(np.int)
df2.dtypes

地址       object
建筑面积    float64
户型       object
租金        int64
dtype: object

In [12]:
df2["mean_prc"] =df2["租金"]/df2["建筑面积"]
df2.dtypes

地址           object
建筑面积        float64
户型           object
租金            int64
mean_prc    float64
dtype: object

In [13]:
df3 = df2.groupby("地址").mean().iloc[:,[0,2]]

In [14]:
df4 = df2.groupby("地址").count().iloc[:,[0]]

In [15]:
df4 = df4.rename(columns={"建筑面积":"count"})

In [16]:
pf5 = pd.merge(df3,df4,left_index = True,right_index=True)
pf5.head()

,建筑面积,mean_prc,count
地址,,,
CBD - 世界城,68.571429,170.160069,14
东城 - FREE自由季,56.142857,92.937408,7
东城 - Naga上院,456.500000,164.204055,2
东城 - solo一期绿景苑,57.333333,76.279156,3
东城 - 万国城,141.750000,141.990849,4


In [20]:
df6 = pd.read_csv("zufang2.csv")

In [21]:
del df6["Unnamed: 0"]
df6 = df6.rename(columns={"name":"地址"}).set_index('地址')
df6.tail(2)

,lon,lat,number,location
地址,,,,
西城 - 丽水莲花二期,116.327634,39.89305,9,西城
西城 - 太平街小区,116.389230,39.87740,22,西城


In [22]:
pf7 = pd.merge(df6,pf5,left_index = True,right_index=True).reset_index().rename(columns={"地址":"name","建筑面积":"mean_area"})
del pf7["count"]
pf7.head()

,name,lon,lat,number,location,mean_area,mean_prc
0,东城 - FREE自由季,116.437104,39.890148,25,东城,56.142857,92.937408
1,东城 - Naga上院,116.428135,39.942008,2,东城,456.500000,164.204055
2,东城 - solo一期绿景苑,116.438224,39.888702,6,东城,57.333333,76.279156
3,东城 - 万国城,116.437920,39.948074,6,东城,141.750000,141.990849
4,东城 - 万国城Moma,116.437920,39.948074,7,东城,143.250000,132.234561


In [23]:
pf7.to_csv("zufang3.csv",encoding="utf-8")

In [24]:
pf7.sort_values("mean_prc")

,name,lon,lat,number,location,mean_area,mean_prc
1200,密云 - 学府花园,116.823124,40.373392,4,密云,1059.000000,9.822034
973,北京周边 - 大运河孔雀城,116.470268,39.908379,2,北京周边,101.000000,10.891089
1263,怀柔 - 益田影人峰汇公寓,116.675638,40.289461,2,怀柔,155.000000,10.967742
1486,房山 - 莱恩小镇,115.984474,39.629870,3,房山,90.000000,11.111111
974,北京周边 - 天悦,116.426248,39.999744,5,北京周边,104.000000,11.538462
1211,密云 - 李各庄小区,116.833604,40.396495,1,密云,127.000000,11.811024
3938,燕郊 - 星河皓月,116.772406,39.980696,14,燕郊,80.300000,12.582984
976,北京周边 - 富力新城,117.003530,39.813424,5,北京周边,84.666667,13.424242
1536,房山 - 韩村河西小区,115.950207,39.602001,1,房山,117.000000,13.675214
1496,房山 - 西山小镇,115.951297,39.674241,4,房山,80.000000,13.750000


In [25]:
pf7["prc_type"] =0

In [26]:
pf7[pf7.mean_prc>0][pf7.mean_prc<100]

,name,lon,lat,number,location,mean_area,mean_prc,prc_type
0,东城 - FREE自由季,116.437104,39.890148,25,东城,56.142857,92.937408,0
2,东城 - solo一期绿景苑,116.438224,39.888702,6,东城,57.333333,76.279156,0
6,东城 - 上龙嘉园,116.406963,39.953746,1,东城,150.000000,94.000000,0
7,东城 - 上龙西里,116.406054,39.952434,56,东城,72.900000,96.246732,0
12,东城 - 东后河沿,116.429038,39.900221,5,东城,117.666667,92.719402,0
16,东城 - 东四八条,116.423334,39.932301,3,东城,30.000000,96.666667,0
21,东城 - 东四十条小区,116.419661,39.933254,2,东城,55.000000,95.454545,0
22,东城 - 东四十条甲34号院,116.427291,39.933143,12,东城,58.000000,91.696608,0
24,东城 - 东四块玉北街小区,116.424134,39.882934,6,东城,60.000000,97.928571,0
25,东城 - 东四块玉南街,116.424977,39.877747,17,东城,73.428571,87.334174,0


In [62]:
pf7.loc[lambda df: 100>df.mean_prc, :].loc[lambda df: df.mean_prc>10, :]

,name,lon,lat,number,location,mean_area,mean_prc,prc_type
0,东城 - FREE自由季,116.437104,39.890148,25,东城,54.320000,92.949698,0
2,东城 - solo一期绿景苑,116.438224,39.888702,6,东城,57.666667,75.208544,0
5,东城 - 三秀涵雅居,116.406724,39.957635,21,东城,85.809524,90.446335,0
6,东城 - 上龙嘉园,116.406963,39.953746,1,东城,150.000000,94.000000,0
7,东城 - 上龙西里,116.406054,39.952434,56,东城,72.571429,97.448063,0
12,东城 - 东后河沿,116.429038,39.900221,5,东城,99.000000,92.251360,0
16,东城 - 东四八条,116.423334,39.932301,3,东城,30.000000,95.555556,0
21,东城 - 东四十条小区,116.419661,39.933254,2,东城,55.000000,95.454545,0
22,东城 - 东四十条甲34号院,116.427291,39.933143,12,东城,58.833333,93.129203,0
24,东城 - 东四块玉北街小区,116.424134,39.882934,6,东城,60.000000,96.095238,0
